In [ ]:
import numpy as np
import os
import pandas as pd
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("/content/sample_data/HipparcosPeriodic.csv")

In [ ]:
display(df)

,hip,cat,type,p2pScatterOnDetrendedTS,p2pScatterOnFoldedTS,scatterOnResidualTS,Raw_WeightedStdDev,Raw_WeightedSkewness,Raw_WeightedKurtosis,Raw_PercentileRange10,stetsonJ,stetsonJweighted,stetsonK,WstetsonJ,WstetsonJweighted,WstetsonK,logPnonQso,logPqso,qsoVar,nonQsoVar,LogPeriod,LogAmplitude,HarmNum,A11,A12,PH12,A13,PH13,A14,PH14,A15,PH15,logA11minusA,logA12_A11,logA13_A12,absGlat,Glat,Glon,Parallax,Absolute_Mag00,BV_Color,VI_Color,JmK,JmH,HmK
0,100033,P,LPV,0.324372,0.052137,0.030846,1.376494,0.314112,-1.538367,-1.99560,63.565943,65.032861,0.842845,63.175113,63.280256,0.924545,-87.325193,-81.779933,55.355023,73.488857,2.481486,0.613493,0,2.054126,0.000000,1.570796,0.000000,0.000000,0.000000,-1.570796,0.000000,3.141593,0.000000,0.000000,0.000000,32.789501,-32.7895,2.0600,7.52,3.686989,1.500,3.00,1.372,0.841,0.531
1,100048,P,LPV,0.436983,0.128058,0.068738,0.890124,0.411059,-1.631588,-1.05040,37.732256,38.920696,0.931126,36.469151,37.231305,0.939286,-36.972470,-66.826823,111.808794,11.642334,2.301030,0.557185,0,1.803821,0.000000,1.570796,0.000000,0.000000,0.000000,-1.570796,0.000000,3.141593,0.000000,0.000000,0.000000,13.245500,13.2455,94.2114,-1.63,0.997338,1.294,2.82,1.181,0.530,0.651
2,100107,P,EB,0.021372,0.002763,0.001121,0.113826,0.606222,-0.576736,-0.09390,5.716224,3.546258,0.843043,5.707364,3.538411,0.841971,-30.103046,-82.084879,41.798619,0.997072,-0.574564,-0.534104,1,0.146119,0.027836,-1.522576,0.000000,0.000000,0.000000,-1.570796,0.000000,3.141593,0.001140,0.075731,0.000000,27.519300,-27.5193,24.2102,7.39,3.362822,0.453,0.52,0.261,0.200,0.061
3,100113,P,LPV,0.024881,0.013072,0.008551,0.417893,0.802179,0.642082,-0.37455,11.015546,10.671530,0.770415,10.847474,10.527592,0.760909,-91.070966,-55.953278,5.816710,27.400029,2.602060,0.173071,3,0.651487,0.105752,-1.258983,0.103804,2.446432,0.037658,0.339756,0.000000,3.141593,0.007620,0.065327,0.297013,0.866200,0.8662,75.4296,-1.41,0.814696,3.000,3.01,2.121,1.290,0.831
4,100135,P,EB,0.046822,0.000995,0.000404,0.249318,0.636493,-0.801945,-0.20540,13.912586,13.450221,0.877788,13.703014,13.278901,0.867624,-47.673088,-133.314043,83.511942,2.509648,-0.025593,-0.118254,8,0.302908,0.108478,-1.579184,0.052260,3.074983,0.026411,1.435496,0.016015,-0.253345,0.010620,0.132939,0.170777,0.203100,0.2031,74.5392,-0.28,-4.082510,0.591,0.66,0.294,0.229,0.065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1656,99765,P,CWA,0.094059,0.046485,0.010558,0.371960,0.945077,-0.057975,-0.26070,10.924088,11.102547,0.798031,10.324198,10.639423,0.764613,-21.016256,-37.487622,31.861380,5.008187,1.454693,0.095423,1,0.504032,0.234946,-2.451889,0.000000,0.000000,0.000000,-1.570796,0.000000,3.141593,0.022044,0.166173,0.000000,24.540400,-24.5404,29.4089,-1.61,1.543929,0.370,0.43,0.510,0.458,0.052
1657,99802,P,LPV,0.188887,0.134721,0.136221,0.990666,0.251523,-1.006608,-1.60220,37.926085,36.861345,0.804452,38.700414,37.389984,0.868575,-26.965243,-32.334677,53.934069,26.642569,2.468521,0.692727,0,2.464329,0.000000,1.570796,0.000000,0.000000,0.000000,-1.570796,0.000000,3.141593,0.000000,0.000000,0.000000,13.948400,-13.9484,50.9999,4.01,2.843222,1.500,3.56,1.526,0.906,0.620
1658,99887,P,DCEP,0.012961,0.003080,0.001443,0.163962,0.114991,-1.481070,-0.19890,5.168745,5.224162,0.912076,5.161610,5.212372,0.911444,-56.549421,-61.060003,5.731234,4.566962,0.827105,-0.354702,6,0.197443,0.043429,3.106008,0.029587,0.155341,0.000000,-1.327667,0.000000,2.660106,0.003380,0.086345,0.225634,0.066700,-0.0667,73.1208,-0.95,0.571018,1.135,1.10,0.882,0.633,0.249
1659,9990,P,ACYG,0.000163,0.000282,0.000136,0.017187,0.037731,-0.063909,-0.02085,1.844442,1.976298,0.792289,1.848616,1.966100,0.792174,-62.092335,-36.183875,0.575434,10.185807,0.429574,-1.417148,0,0.019137,0.000000,1.570796,0.000000,0.000000,0.000000,-1.570796,0.000000,3.141593,0.000000,0.000000,0.000000,2.942900,-2.9429,132.9163,1.79,-2.923335,

Variables that we need to select are

attr.to.choose= c("LogPeriod","JmK","Raw_PercentileRange10", "JmH","VI_Color","Raw_WeightedSkewness",   
                  "BV_Color","A11","LogAmplitude","Raw_WeightedStdDev",
                  "nonQsoVar","logA12_A11","qsoVar","WstetsonK","PH12",
                  "p2pScatterOnDetrendedTS")

In [ ]:
attr_to_choose=["LogPeriod","JmK","Raw_PercentileRange10", "JmH","VI_Color","Raw_WeightedSkewness",
"BV_Color","A11","LogAmplitude","Raw_WeightedStdDev", "nonQsoVar","logA12_A11","qsoVar","WstetsonK","PH12", "p2pScatterOnDetrendedTS"]

In [ ]:
#Convert to explanatory and response variables
x=df[attr_to_choose].values
dummies = pd.get_dummies(df['type']) # Classification
types = dummies.columns
y = dummies.values
display(y)



array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
display(x)
display(y)

array([[ 2.48148606e+00,  1.37200000e+00, -1.99560000e+00, ...,
         9.24545475e-01,  1.57079633e+00,  3.24371721e-01],
       [ 2.30103000e+00,  1.18100000e+00, -1.05040000e+00, ...,
         9.39285577e-01,  1.57079633e+00,  4.36982819e-01],
       [-5.74563676e-01,  2.61000000e-01, -9.39000000e-02, ...,
         8.41971033e-01, -1.52257629e+00,  2.13716252e-02],
       ...,
       [ 8.27105302e-01,  8.82000000e-01, -1.98900000e-01, ...,
         9.11443968e-01,  3.10600844e+00,  1.29606783e-02],
       [ 4.29573822e-01,  4.16000000e-01, -2.08500000e-02, ...,
         7.92173723e-01,  1.57079633e+00,  1.62766379e-04],
       [ 1.22621356e+00, -9.20000000e-02, -3.25500000e-02, ...,
         8.07344853e-01,  2.02667644e+00,  2.70930480e-04]])

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import regularizers

# np.argmax(pred,axis=1)
# Cross-validate
# Use for StratifiedKFold classification
kf = StratifiedKFold(5, shuffle=True, random_state=42) 
    
oos_y = []
oos_pred = []
fold = 0

# Must specify y StratifiedKFold for
for train, test in kf.split(x,df['type']):  
    fold+=1
    print(f"Fold #{fold}")
        
    x_train = x[train]
    y_train = y[train]
    x_test = x[test]
    y_test = y[test]
    
    model = Sequential()
    model.add(Dense(32, input_dim=x.shape[1], 
                    activation='relu',
                    #kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),
                    activity_regularizer=regularizers.l1(1e-2))) # Hidden 1
    model.add(Dense(96, activation='relu',
                    activity_regularizer=regularizers.l1(1e-2))) # Hidden 2
    model.add(Dense(units=df['type'].nunique(),activation="softmax")) # Output
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    model.fit(x_train,y_train,validation_data=(x_test,y_test),verbose=0,epochs=500)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    # raw probabilities to chosen class (highest probability)
    pred = np.argmax(pred,axis=1) 
    oos_pred.append(pred)  

    # Measure this fold's accuracy
    y_compare = np.argmax(y_test,axis=1) # For accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print(f"Fold score (accuracy): {score}")

# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
oos_y_compare = np.argmax(oos_y,axis=1) # For accuracy calculation

score = metrics.accuracy_score(oos_y_compare, oos_pred)
print(f"Final score (accuracy): {score}")    
    
# Write the cross-validated prediction
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )

Fold #1
Fold score (accuracy): 0.7207207207207207
Fold #2
Fold score (accuracy): 0.7650602409638554
Fold #3
Fold score (accuracy): 0.7439759036144579
Fold #4
Fold score (accuracy): 0.7530120481927711
Fold #5
Fold score (accuracy): 0.7228915662650602
Final score (accuracy): 0.741119807344973


I tried to run the code several times and found that the Fold #2 consistently gives higher score. So it is clear that there is an outlier in Fold #2, which might be causing this. We could look into it, to see if we want to keep the outlier or at the very least we want to know about it.

# Drop Out for Keras to Decrease Overfitting 

In [ ]:
import pandas as pd
from scipy.stats import zscore

# Read the data set
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])

# Generate dummies for job
df = pd.concat([df,pd.get_dummies(df['job'],prefix="job")],axis=1)
df.drop('job', axis=1, inplace=True)

# Generate dummies for area
df = pd.concat([df,pd.get_dummies(df['area'],prefix="area")],axis=1)
df.drop('area', axis=1, inplace=True)

# Missing values for income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standardize ranges
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['age'] = zscore(df['age'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convert to numpy - Classification
x_columns = df.columns.drop('product').drop('id')
x = df[x_columns].values
dummies = pd.get_dummies(df['product']) # Classification
products = dummies.columns
y = dummies.values


In [ ]:
########################################
# Keras with dropout for Classification
########################################

import pandas as pd
import os
import numpy as np
from sklearn import metrics
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras import regularizers


kf=KFold(5,shuffle=True,random_state=42)
oos_y=[]
oos_pred=[]
fold=0

for train, test in kf.split(x):
    fold+=1
    print(f"Fold #{fold}")

    x_train=x[train]
    y_train=y[train]
    x_test=x[test]
    y_test=y[test]

    model = Sequential()
    model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
    model.add(Dropout(0.1))
    model.add(Dense(25, activation='relu', \
                activity_regularizer=regularizers.l1(1e-4))) # Hidden 2
    # Usually do not add dropout after final hidden layer
    #model.add(Dropout(0.5)) 
    model.add(Dense(y.shape[1],activation='softmax')) # Output
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    model.fit(x_train,y_train,validation_data=(x_test,y_test),\
              verbose=0,epochs=500)
    
    pred = model.predict(x_test)
    
    oos_y.append(y_test)
    # raw probabilities to chosen class (highest probability)
    pred = np.argmax(pred,axis=1) 
    oos_pred.append(pred)        

    # Measure this fold's accuracy
    y_compare = np.argmax(y_test,axis=1) # For accuracy calculation
    score = metrics.accuracy_score(y_compare, pred)
    print(f"Fold score (accuracy): {score}")


# Build the oos prediction list and calculate the error.
oos_y = np.concatenate(oos_y)
oos_pred = np.concatenate(oos_pred)
oos_y_compare = np.argmax(oos_y,axis=1) # For accuracy calculation

score = metrics.accuracy_score(oos_y_compare, oos_pred)
print(f"Final score (accuracy): {score}")    
    
# Write the cross-validated prediction
oos_y = pd.DataFrame(oos_y)
oos_pred = pd.DataFrame(oos_pred)
oosDF = pd.concat( [df, oos_y, oos_pred],axis=1 )
#oosDF.to_csv(filename_write,index=False)

Fold #1
Fold score (accuracy): 0.655
Fold #2
Fold score (accuracy): 0.6975
Fold #3
Fold score (accuracy): 0.7075
Fold #4
Fold score (accuracy): 0.7025
Fold #5
Fold score (accuracy): 0.655
Final score (accuracy): 0.6835


It seems that the dropout regularization seems to decrease the overall accuracy